## Zero-shot without classes incoporated

In [ ]:
import torch
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel
import os
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix, BinaryPrecisionRecallCurve

def clip_pred(imgs):
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    inputs = processor(
        text=["a synthetic image created by AI", "a real image taken by a human"],
        images=imgs,
        return_tensors="pt",
        padding=True
    )

    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
    prob = logits_per_image.softmax(dim=1)  
    return prob

In [ ]:
probs = clip_pred(images)
probs[:,1]

In [ ]:
def load_images_from_folder(folder):
    images = []
    
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
        
    
    labels = torch.zeros(len(images), dtype=torch.int)
    return images, labels

In [ ]:
def metrics(ys, ts):
    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    acc.update(ys, ts)
    f1.update(ys, ts)
    cm.update(ys, ts)
    return acc.compute(), f1.compute(), cm.compute()


In [ ]:
folder_path = r'C:\Users\vikto\Documents\GitHub\DTU_repo\deep_learning\4_Convolutional\images'
images, labels = load_images_from_folder(folder_path)
probs = clip_pred(images)
metrics(probs[:,1], labels)

### HPC adjusted zero-shot no classes

In [ ]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import os
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix

# Enable debugging
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["NCCL_DEBUG"] = "INFO"
os.environ["PYTHONFAULTHANDLER"] = "1"

# Log GPU details
if torch.cuda.is_available():
    print(f"CUDA is available. Device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Memory Allocated: {torch.cuda.memory_allocated() / (1024 ** 3):.2f} GB")
    print(f"CUDA Memory Reserved: {torch.cuda.memory_reserved() / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available. Check your GPU setup.")

def clip_pred(imgs, model, processor):
    """
    Perform prediction using the CLIP model.
    """
    inputs = processor(
        text=["a synthetic image created by AI", "a real image taken by a human"],
        images=imgs,
        return_tensors="pt",
        padding=True
    )
    
    inputs = {k: v.to("cuda") for k, v in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image  # Image-text similarity score
    prob = logits_per_image.softmax(dim=1)  # Probability over classes
    return prob

def load_images_from_folders(fake_folder, real_folder):
    """
    Load images and their corresponding labels from specified folders.
    """
    images = []
    labels = []
    
    # Load FAKE images
    for filename in os.listdir(fake_folder):
        img_path = os.path.join(fake_folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path).convert("RGB")
            images.append(img)
            labels.append(0)  # Label for "FAKE"
    
    # Load REAL images
    for filename in os.listdir(real_folder):
        img_path = os.path.join(real_folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path).convert("RGB")
            images.append(img)
            labels.append(1)  # Label for "REAL"
    
    return images, torch.tensor(labels, dtype=torch.int)

def evaluate_model(images, labels, batch_size=64):
    """
    Evaluate the CLIP model using mini-batch processing and calculate metrics.
    """
    # Load the CLIP model and processor
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cuda")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    
    # Process images in mini-batches
    probs = []
    for i in range(0, len(images), batch_size):
        batch_imgs = images[i:i+batch_size]
        batch_probs = clip_pred(batch_imgs, model, processor)
        probs.append(batch_probs)
    
    probs = torch.cat(probs, dim=0)  # Combine all batches
    preds = torch.argmax(probs, dim=1)  # Predicted labels
    
    # Calculate metrics
    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    
    acc.update(preds, labels)
    f1.update(preds, labels)
    cm.update(preds, labels)
    
    print(f"Accuracy: {acc.compute().item():.4f}")
    print(f"F1 Score: {f1.compute().item():.4f}")
    print(f"Confusion Matrix: {cm.compute()}")
    return acc.compute(), f1.compute(), cm.compute()

# Paths to the datasets
fake_folder = r'/dtu/blackhole/18/160664/test/FAKE/'
real_folder = r'/dtu/blackhole/18/160664/test/REAL/'

# Load the images and labels
images, labels = load_images_from_folders(fake_folder, real_folder)
print(f"Loaded {len(images)} images.")
print(f"Labels: {labels}")

# Evaluate the model
evaluate_model(images, labels)

## Zero-shot with classes implemented

In [ ]:
import torch
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel
import os
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix, BinaryPrecisionRecallCurve

def clip_pred(imgs):
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    logits = torch.empty(0,2)

    for i in imgs:
        img_class = imgs[i]
        input = processor(
            text=["a artificially generated image of a" + str(i), "a real image of a" + str(i)],
            images=img_class,
            return_tensors="pt",
            padding=True
        )

        output = model(**input)
        logits = torch.cat((logits, output.logits_per_image), dim=0)  # this is the image-text similarity score
    prob = logits.softmax(dim=1)  
    return prob


def load_images_from_folder(folder):
    images = {'airplane': [], 'automobile': [], 'bird': [], 'cat': [], 'deer': [], 'dog': [], 'frog': [], 'horse': [], 'ship': [], 'truck': []}
    class_types = {'(2)':'automobile','(3)': 'bird', '(4)': 'cat', '(5)': 'deer', '(6)': 'dog', '(7)': 'frog', '(8)': 'horse', '(9)': 'ship', '(10)': 'truck'}

    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename))
        for key, value in class_types.items():
                if key in filename:
                     img_index = value
                     break
                else:
                     img_index = 'airplane'
        images[img_index].append(img)
            

    labels = torch.zeros(len(images), dtype=torch.int)
    return images, labels

def metrics(ys, ts):
    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    prc = BinaryPrecisionRecallCurve()
    acc.update(ys, ts)
    f1.update(ys, ts)
    cm.update(ys, ts)
    prc.update(ys, ts)

    return acc.compute(), f1.compute(), cm.compute(), prc.compute()


folder_path = r'C:\Users\vikto\Documents\GitHub\Feature-analysis-using-CLIP-model\images'
images, labels = load_images_from_folder(folder_path)
# probs = clip_pred(images)
# metrics(probs[:,1], labels)


### 

### Non HPC adjusted zero-shot with classes

In [ ]:
import torch
from PIL import Image
import os
from transformers import CLIPProcessor, CLIPModel
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix, BinaryAUROC

def clip_pred(imgs, imgs_class):
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    # Create prompts for all images
    prompts = []
    for img_class in imgs_class:
        prompts.append("a synthetic image of a " + str(img_class))
        prompts.append("a real image of a " + str(img_class))

    # Process all images and prompts in a single batch
    inputs = processor(
        text=prompts,
        images=imgs * 2,  # Duplicate images to match the number of prompts
        return_tensors="pt",
        padding=True
    )

    # Pass the batch to the model
    outputs = model(**inputs)
    logits = outputs.logits_per_image  # this is the image-text similarity score

    # Reshape logits to separate synthetic and real image logits
    logits = logits.view(len(imgs), 2, -1).mean(dim=2)

    prob = logits.softmax(dim=1)
    return prob

def load_images_from_folder(folder):
    images = []
    class_types = {'(2)': 'automobile', '(3)': 'bird', '(4)': 'cat', '(5)': 'deer', '(6)': 'dog', '(7)': 'frog', '(8)': 'horse', '(9)': 'ship', '(10)': 'truck'}
    imgs_class = []

    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename))
        if img is not None:
            images.append(img)

            for c in class_types.keys():
                if c in filename:
                    imgs_class.append(class_types.get(c))
                    break
            else:
                imgs_class.append('airplane')

    labels = torch.zeros(len(images), dtype=torch.int)
    return images, labels, imgs_class

def metrics(ys, ts):
    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    auroc = BinaryAUROC()
    acc.update(ys, ts)
    f1.update(ys, ts)
    cm.update(ys, ts)
    auroc.update(ys, ts)

    return acc.compute(), f1.compute(), cm.compute(), auroc.compute()

folder_path = r'C:\Users\vikto\Documents\GitHub\Feature-analysis-using-CLIP-model\images'
images, labels, imgs_class = load_images_from_folder(folder_path)
probs = clip_pred(images, imgs_class)
metrics(probs[:, 1], labels)

In [ ]:
import torch
from PIL import Image
import os
from transformers import CLIPProcessor, CLIPModel
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix, BinaryAUROC
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve


def clip_pred(imgs, imgs_class):
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    # Create prompts for all images
    prompts = []
    for img_class in imgs_class:
        prompts.append("a synthetic image of a " + str(img_class))
        prompts.append("a real image of a " + str(img_class))

    # Process all images and prompts in a single batch
    inputs = processor(
        text=prompts,
        images=imgs * 2,  # Duplicate images to match the number of prompts
        return_tensors="pt",
        padding=True
    )

    # Pass the batch to the model
    outputs = model(**inputs)
    logits = outputs.logits_per_image  # this is the image-text similarity score

    # Reshape logits to separate synthetic and real image logits
    logits = logits.view(len(imgs), 2, -1).mean(dim=2)

    prob = logits.softmax(dim=1)
    return prob

def load_images_from_folder(folder):
    images = []
    class_types = {'(2)': 'automobile', '(3)': 'bird', '(4)': 'cat', '(5)': 'deer', '(6)': 'dog', '(7)': 'frog', '(8)': 'horse', '(9)': 'ship', '(10)': 'truck'}
    imgs_class = []

    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename))
        if img is not None:
            images.append(img)

            for c in class_types.keys():
                if c in filename:
                    imgs_class.append(class_types.get(c))
                    break
            else:
                imgs_class.append('airplane')

    labels = torch.zeros(len(images), dtype=torch.int)
    return images, labels, imgs_class

def evaluate_metrics(probs, labels):
    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    auroc = BinaryAUROC()

    acc.update(probs[:, 1], labels)
    f1.update(probs[:, 1], labels)
    cm.update(probs[:, 1], labels)
    auroc.update(probs[:, 1], labels)

    accuracy = acc.compute()
    f1_score = f1.compute()
    confusion_matrix = cm.compute()
    auroc_score = auroc.compute()

    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1_score}")
    print(f"Confusion Matrix: \n{confusion_matrix}")
    print(f"AUROC: {auroc_score}")

    # Plot ROC Curve
    fpr, tpr, _ = roc_curve(labels.detach().numpy(), probs[:, 1].detach().numpy())
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, marker='.', label='ROC Curve (AUC = {:.2f})'.format(auroc_score))
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.grid(True)
    plt.show()

folder_path = r'C:\Users\vikto\Documents\GitHub\Feature-analysis-using-CLIP-model\images'
images, labels, imgs_class = load_images_from_folder(folder_path)
probs = clip_pred(images, imgs_class)
evaluate_metrics(probs, labels)

### Adjusted class code for HPC

#### For running fake and real seperately:

In [ ]:
import torch
from PIL import Image
import os
from transformers import CLIPProcessor, CLIPModel
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix, BinaryAUROC

# Enable debugging
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["NCCL_DEBUG"] = "INFO"
os.environ["PYTHONFAULTHANDLER"] = "1"

# Log GPU details
if torch.cuda.is_available():
    print(f"CUDA is available. Device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Memory Allocated: {torch.cuda.memory_allocated() / (1024 ** 3):.2f} GB")
    print(f"CUDA Memory Reserved: {torch.cuda.memory_reserved() / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available. Check your GPU setup.")

def clip_pred(imgs, imgs_class, model, processor):
    """
    Perform prediction using the CLIP model.
    """
    # Create prompts for all images
    prompts = []
    for img_class in imgs_class:
        prompts.append("a synthetic image of a " + str(img_class))
        prompts.append("a real image of a " + str(img_class))

    # Process all images and prompts in a single batch
    inputs = processor(
        text=prompts,
        images=imgs * 2,  # Duplicate images to match the number of prompts
        return_tensors="pt",
        padding=True
    )

    inputs = {k: v.to("cuda") for k, v in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)
    logits = outputs.logits_per_image  # this is the image-text similarity score

    # Reshape logits to separate synthetic and real image logits
    logits = logits.view(len(imgs), 2, -1).mean(dim=2)

    prob = logits.softmax(dim=1)
    return prob

def load_images_from_folder(folder):
    images = []
    class_types = {'(2)': 'automobile', '(3)': 'bird', '(4)': 'cat', '(5)': 'deer', '(6)': 'dog', '(7)': 'frog', '(8)': 'horse', '(9)': 'ship', '(10)': 'truck'}
    imgs_class = []

    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename)).convert("RGB")
        if img is not None:
            images.append(img)

            for c in class_types.keys():
                if c in filename:
                    imgs_class.append(class_types.get(c))
                    break
            else:
                imgs_class.append('airplane')

    labels = torch.zeros(len(images), dtype=torch.int)
    return images, labels, imgs_class

def evaluate_model(images, labels, imgs_class, batch_size=64):
    """
    Evaluate the CLIP model using mini-batch processing and calculate metrics.
    """
    # Load the CLIP model and processor
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cuda")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    
    # Process images in mini-batches
    probs = []
    for i in range(0, len(images), batch_size):
        batch_imgs = images[i:i+batch_size]
        batch_classes = imgs_class[i:i+batch_size]
        batch_probs = clip_pred(batch_imgs, batch_classes, model, processor)
        probs.append(batch_probs)
    
    probs = torch.cat(probs, dim=0)  # Combine all batches
    preds = torch.argmax(probs, dim=1)  # Predicted labels
    
    # Calculate metrics
    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    auroc = BinaryAUROC()
    
    acc.update(preds, labels)
    cm.update(preds, labels)
    auroc.update(preds, labels)
    
    accuracy = acc.compute()
    confusion_matrix = cm.compute()
    auroc_score = auroc.compute()
    
    print(f"Accuracy: {accuracy}")
    print(f"Confusion Matrix: \n{confusion_matrix}")
    print(f"AUROC: {auroc_score}")

# Paths to the datasets
folder_path = r'/dtu/blackhole/18/160664/test/REAL/'

# Load the images and labels
images, labels, imgs_class = load_images_from_folder(folder_path)
print(f"Loaded {len(images)} images.")
print(f"Labels: {labels}")

# Evaluate the model
evaluate_model(images, labels, imgs_class)

#### Fake and real at the same time:

In [ ]:
import torch
from PIL import Image
import os
from transformers import CLIPProcessor, CLIPModel
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix, BinaryAUROC

# Enable debugging
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["NCCL_DEBUG"] = "INFO"
os.environ["PYTHONFAULTHANDLER"] = "1"

# Log GPU details
if torch.cuda.is_available():
    print(f"CUDA is available. Device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Memory Allocated: {torch.cuda.memory_allocated() / (1024 ** 3):.2f} GB")
    print(f"CUDA Memory Reserved: {torch.cuda.memory_reserved() / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available. Check your GPU setup.")

def clip_pred(imgs, imgs_class, model, processor):
    """
    Perform prediction using the CLIP model.
    """
    # Create prompts for all images
    prompts = []
    for img_class in imgs_class:
        prompts.append("a synthetic image of a " + str(img_class))
        prompts.append("a real image of a " + str(img_class))

    # Process all images and prompts in a single batch
    inputs = processor(
        text=prompts,
        images=imgs * 2,  # Duplicate images to match the number of prompts
        return_tensors="pt",
        padding=True
    )

    inputs = {k: v.to("cuda") for k, v in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)
    logits = outputs.logits_per_image  # this is the image-text similarity score

    # Reshape logits to separate synthetic and real image logits
    logits = logits.view(len(imgs), 2, -1).mean(dim=2)

    prob = logits.softmax(dim=1)
    return prob

def load_images_from_folders(fake_folder, real_folder):
    images = []
    labels = []
    class_types = {'(2)': 'automobile', '(3)': 'bird', '(4)': 'cat', '(5)': 'deer', '(6)': 'dog', '(7)': 'frog', '(8)': 'horse', '(9)': 'ship', '(10)': 'truck'}
    imgs_class = []

    # Load FAKE images
    for filename in os.listdir(fake_folder):
        img_path = os.path.join(fake_folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path).convert("RGB")
            images.append(img)
            labels.append(0)  # Label for "FAKE"

            for c in class_types.keys():
                if c in filename:
                    imgs_class.append(class_types.get(c))
                    break
            else:
                imgs_class.append('airplane')

    # Load REAL images
    for filename in os.listdir(real_folder):
        img_path = os.path.join(real_folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path).convert("RGB")
            images.append(img)
            labels.append(1)  # Label for "REAL"

            for c in class_types.keys():
                if c in filename:
                    imgs_class.append(class_types.get(c))
                    break
            else:
                imgs_class.append('airplane')

    return images, torch.tensor(labels, dtype=torch.int), imgs_class

def evaluate_model(images, labels, imgs_class, batch_size=64):
    """
    Evaluate the CLIP model using mini-batch processing and calculate metrics.
    """
    # Load the CLIP model and processor
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cuda")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    
    # Process images in mini-batches
    probs = []
    for i in range(0, len(images), batch_size):
        batch_imgs = images[i:i+batch_size]
        batch_classes = imgs_class[i:i+batch_size]
        batch_probs = clip_pred(batch_imgs, batch_classes, model, processor)
        probs.append(batch_probs)
    
    probs = torch.cat(probs, dim=0)  # Combine all batches
    preds = torch.argmax(probs, dim=1)  # Predicted labels
    
    # Calculate metrics
    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    auroc = BinaryAUROC()
    
    acc.update(preds, labels)
    f1.update(preds, labels)
    cm.update(preds, labels)
    auroc.update(preds, labels)
    
    accuracy = acc.compute()
    f1_score = f1.compute()
    confusion_matrix = cm.compute()
    auroc_score = auroc.compute()
    
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1_score}")
    print(f"Confusion Matrix: \n{confusion_matrix}")
    print(f"AUROC: {auroc_score}")

# Paths to the datasets
fake_folder = r'/dtu/blackhole/18/160664/test/FAKE/'
real_folder = r'/dtu/blackhole/18/160664/test/REAL/'

# Load the images and labels
images, labels, imgs_class = load_images_from_folders(fake_folder, real_folder)
print(f"Loaded {len(images)} images.")
print(f"Labels: {labels}")

# Evaluate the model
evaluate_model(images, labels, imgs_class)

### Extracting image features and applying FFNN

In [ ]:
import torch
from PIL import Image
import os
from transformers import CLIPProcessor, CLIPModel
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix, BinaryAUROC

# Enable debugging
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["NCCL_DEBUG"] = "INFO"
os.environ["PYTHONFAULTHANDLER"] = "1"

# Log GPU details
if torch.cuda.is_available():
    print(f"CUDA is available. Device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Memory Allocated: {torch.cuda.memory_allocated() / (1024 ** 3):.2f} GB")
    print(f"CUDA Memory Reserved: {torch.cuda.memory_reserved() / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available. Check your GPU setup.")

def clip_pred(imgs, imgs_class, model, processor):
    """
    Perform prediction using the CLIP model.
    """
    # Create prompts for all images
    prompts = []
    for img_class in imgs_class:
        prompts.append("a synthetic image of a " + str(img_class))
        prompts.append("a real image of a " + str(img_class))

    # Process all images and prompts in a single batch
    inputs = processor(
        text=prompts,
        images=imgs * 2,  # Duplicate images to match the number of prompts
        return_tensors="pt",
        padding=True
    )

    inputs = {k: v.to("cuda") for k, v in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)
        img_features = model.get_image_features(pixel_values = inputs['pixel_values'], output_hidden_states = False)
    logits = outputs.logits_per_image  # this is the image-text similarity score

    # Reshape logits to separate synthetic and real image logits
    logits = logits.view(len(imgs), 2, -1).mean(dim=2)

    prob = logits.softmax(dim=1)
    return prob, img_features

def load_images_from_folders(fake_folder, real_folder):
    images = []
    labels = []
    class_types = {'(2)': 'automobile', '(3)': 'bird', '(4)': 'cat', '(5)': 'deer', '(6)': 'dog', '(7)': 'frog', '(8)': 'horse', '(9)': 'ship', '(10)': 'truck'}
    imgs_class = []

    # Load FAKE images
    for filename in os.listdir(fake_folder):
        img_path = os.path.join(fake_folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path).convert("RGB")
            images.append(img)
            labels.append(0)  # Label for "FAKE"

            for c in class_types.keys():
                if c in filename:
                    imgs_class.append(class_types.get(c))
                    break
            else:
                imgs_class.append('airplane')

    # Load REAL images
    for filename in os.listdir(real_folder):
        img_path = os.path.join(real_folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path).convert("RGB")
            images.append(img)
            labels.append(1)  # Label for "REAL"

            for c in class_types.keys():
                if c in filename:
                    imgs_class.append(class_types.get(c))
                    break
            else:
                imgs_class.append('airplane')

    return images, torch.tensor(labels, dtype=torch.int), imgs_class

def evaluate_model(images, labels, imgs_class, batch_size=64):
    """
    Evaluate the CLIP model using mini-batch processing and calculate metrics.
    """
    # Load the CLIP model and processor
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cuda")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    
    # Process images in mini-batches
    probs = []
    image_features = []
    for i in range(0, len(images), batch_size):
        batch_imgs = images[i:i+batch_size]
        batch_classes = imgs_class[i:i+batch_size]
        batch_probs, batch_features = clip_pred(batch_imgs, batch_classes, model, processor)
        probs.append(batch_probs)
        image_features.append(batch_features)
    
    probs = torch.cat(probs, dim=0)  # Combine all batches
    preds = torch.argmax(probs, dim=1)  # Predicted labels
    image_features = torch.cat(batch_features, dim=0)
    #TODO torch.save?
    
    # Calculate metrics
    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    auroc = BinaryAUROC()
    
    acc.update(preds, labels)
    f1.update(preds, labels)
    cm.update(preds, labels)
    auroc.update(preds, labels)
    
    accuracy = acc.compute()
    f1_score = f1.compute()
    confusion_matrix = cm.compute()
    auroc_score = auroc.compute()
    
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1_score}")
    print(f"Confusion Matrix: \n{confusion_matrix}")
    print(f"AUROC: {auroc_score}")

    #Create dataset with labels and features for FFNN
    #TODO


# Paths to the datasets
fake_folder = r'/dtu/blackhole/18/160664/test/FAKE/'
real_folder = r'/dtu/blackhole/18/160664/test/REAL/'

# Load the images and labels
images, labels, imgs_class = load_images_from_folders(fake_folder, real_folder)
print(f"Loaded {len(images)} images.")
print(f"Labels: {labels}")

# Evaluate the model
evaluate_model(images, labels, imgs_class)

#### Train test split:

#### Feed foward neural network on image features from encoder:

In [ ]:
from torch import nn

class network(nn.Module):

    def __init__(self, )